In [89]:
import random
import json
import pandas as pd

class WeightedRandomizer:
    def __init__ (self, weights):
        self.__max = .0
        self.__weights = []
        for value, weight in weights.items ():
            self.__max += weight
            self.__weights.append ( (self.__max, value) )

    def random (self):
        r = random.random () * self.__max
        for ceil, value in self.__weights:
            if ceil > r: return value
            
w = { "Legendary": 0.75, "Epic": 3.76, "Rare": 16.9, "Common": 78.59}
wr = WeightedRandomizer(w)

def pick_rarities():
    return [wr.random() if i not in [0,14,29] else "Rare" for i in range(30)]

with open("AllSets.enUS.json", 'r') as fp:
    all = json.load(fp)
    
# don't load cards from these sets, they aren't used in Arena
excluded_sets = ["Hero Skins", "Missions", "System", "Credits", "Debug", "Promotion", "Tavern Brawl", "Reward"]

arena_sets = [s for s in all.keys() if s not in excluded_sets]

#since many fields are missing, we first need to build a set of valid columns names
column_names = set()
for set_name in arena_sets:
    for card in all[set_name]:
        column_names = column_names.union(set(card.keys()))

column_names.add("set")

card_values = dict()
for col in column_names:
    card_values[col] = []
    
for set_name in arena_sets:
    for card in all[set_name]:
        for col in column_names:
            if col == "set":
                card_values["set"].append(set_name)
            else:
                card_values[col].append(card.get(col, None))

cards_df = pd.DataFrame(data=card_values)
# TODO: Set weight to 1.25 for LOE when LOE bonus goes into effect
cards_df["draft_weight"] = 1 

# "Free" cards should be "Common"
cards_df[cards_df.rarity == "Free"] = "Common"

# TODO: Remove wing 3 and 4 LOE cards
cards_df = cards_df.set_index("id")

# drop any non-real cards
cards_df = cards_df.dropna(subset=["cost"])
cards_df = cards_df[cards_df.collectible == True]

player_classes = [u'Mage', u'Paladin', u'Warlock', u'Warrior', u'Hunter', u'Priest', u'Shaman', u'Druid', u'Rogue']
draftables = dict()

for c in player_classes:
    draftables[c] = cards_df[ (cards_df.playerClass == c) | (cards_df.playerClass.isnull())]
    

def draft_options(playerClass, rarity):
    possible = draftables[playerClass]
    possible = possible[possible.rarity == rarity]
    picks = []
    for i in range(3):
        picks.append(possible.sample(weights="draft_weight").iloc[0])
    return picks

def draft_picks(playerClass):
    rarities = pick_rarities()
    picks = []
    for rarity in rarities:
        picks.append(draft_options(playerClass, rarity))
    return picks

    
cards_df.describe()

,artist,attack,collectible,cost,durability,elite,faction,flavor,health,howToGet,...,inPlayText,mechanics,name,playerClass,race,rarity,set,text,type,draft_weight
count,680,493,680,680,21,91,264,679,472,106,...,25,393,680,388,155,680,680,665,680,680
unique,226,12,1,13,6,2,3,679,12,65,...,24,41,680,9,7,4,7,617,3,1
top,Alex Horley Orlandelli,2,True,3,2,True,Neutral,"If she had studied harder, she would have been...",4,Unlocked at Level 4.,...,Summoning,[Battlecry],Ice Block,Hunter,Mech,Common,Classic,<b>Taunt</b>,Minion,1
freq,38,110,680,138,11,89,213,1,95,9,...,2,152,1,44,48,315,245,14,472,680


In [95]:
options = draft_picks("Mage")
[ {row[0].rarity: [c["name"] for c in row] } for row in options]

[{u'Rare': [u'Crazed Alchemist', u'Demolisher', u'Kirin Tor Mage']},
 {u'Common': [u'Stormpike Commando', u'Clockwork Gnome', u'Young Dragonhawk']},
 {u'Common': [u'Mechanical Yeti', u'Goldshire Footman', u'Razorfen Hunter']},
 {u'Common': [u'Piloted Shredder', u'Fossilized Devilsaur', u'Spellslinger']},
 {u'Common': [u'Mana Wyrm', u'Ice Rager', u'Frostwolf Grunt']},
 {u'Rare': [u'Bloodsail Corsair', u'Sunwalker', u'Counterspell']},
 {u'Common': [u'Dire Wolf Alpha', u'Dire Wolf Alpha', u'Amani Berserker']},
 {u'Common': [u'Murloc Tinyfin', u'Loot Hoarder', u'Clockwork Knight']},
 {u'Common': [u'Frostbolt', u'Scarlet Crusader', u'Ethereal Conjurer']},
 {u'Common': [u'Gurubashi Berserker', u'Flame Juggler', u'Jungle Panther']},
 {u'Epic': [u'Coldarra Drake', u'Fel Reaver', u'Coldarra Drake']},
 {u'Rare': [u'Master Swordsmith', u'Coldlight Seer', u'Fencing Coach']},
 {u'Common': [u'Evil Heckler', u'Bloodsail Raider', u'Tauren Warrior']},
 {u'Common': [u'Ancient Brewmaster',
   u'Scarlet C